In [1]:
cd /Users/bastus/Desktop/Forschungsprojekt/genimpro

/Users/bastus/Desktop/Forschungsprojekt/genimpro


In [2]:
%run importwrapper.py
%pylab inline
samplerate=44100.0

/Library/Python/2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


ImportError: dlopen(/usr/local/lib/python2.7/site-packages/essentia/_essentia.so, 2): Library not loaded: /usr/local/lib/libavformat.56.dylib
  Referenced from: /usr/local/lib/python2.7/site-packages/essentia/_essentia.so
  Reason: image not found

Populating the interactive namespace from numpy and matplotlib


In [5]:
import os.path, time
from datetime import datetime
import sqlite3 as lite
db = lite.connect('genImpro.db')
c = db.cursor() 

def updateDateForRecording(recordingID):
    f = '%Y-%m-%d %H:%M:%S'
    
    dt_obj = datetime.fromtimestamp(os.path.getmtime(recordingDetails[5])).strftime(f)
    
    sqlcommand = "Update recordings set date=%s where ID = %i" % (repr(dt_obj),recordingID)
    #print sqlcommand
    c.execute(sqlcommand)
    db.commit()
    
    #print "last modified: %s" % time.ctime(os.path.getmtime(recordingDetails[5]).strftime(f))
    #print "created: %s" % time.ctime(os.path.getctime(recordingDetails[5]).strftime(f))
    

In [6]:
def analyseTrack(track,updateAll=0):
    
    audio = genimpro.recordings.getAudioForTrack(track)
    sonicevents = genimpro.recordings.getSoniceventsForTrack(track)
    sequences = genimpro.recordings.getSequencesForTrack(track)
    phenotypes = genimpro.recordings.getPhenotypesForTrack(track)
    genotypes = genimpro.recordings.getGenotypesForTrack(track)
    
    if (sonicevents==None) | updateAll:
        sonicevents = genimpro.soniceventsForTrack(track)
        genimpro.recordings.updateSoniceventsForTrack(track,sonicevents)
        
        sequences = genimpro.grouping.sequencesForSonicevents(sonicevents)
        genimpro.recordings.updateSequencesForTrack(track,sequences,sonicevents)
        
        phenotypes = genimpro.phenotypesForSequences(sequences,sonicevents,audio)
        genimpro.recordings.updatePhenotypesForTrack(track,sequences,phenotypes)

        genotypes = genimpro.genotypesForSequences(phenotypes)
        genimpro.recordings.updateGenotypesForTrack(track,sequences,genotypes)

        return
        
    if (sequences==None) | updateAll:
        sequences = genimpro.grouping.sequencesForSonicevents(sonicevents)
        genimpro.recordings.updateSequencesForTrack(track,sequences,sonicevents)
    
        phenotypes = genimpro.phenotypesForSequences(sequences,sonicevents,audio)
        genimpro.recordings.updatePhenotypesForTrack(track,sequences,phenotypes)

        genotypes = genimpro.genotypesForSequences(phenotypes)
        genimpro.recordings.updateGenotypesForTrack(track,sequences,genotypes)
        
        return
        
    if (phenotypes==None) | updateAll:
        phenotypes = genimpro.phenotypesForSequences(sequences,sonicevents,audio)
        genimpro.recordings.updatePhenotypesForTrack(track,sequences,phenotypes)   

        genotypes = genimpro.genotypesForSequences(phenotypes)
        genimpro.recordings.updateGenotypesForTrack(track,sequences,genotypes)

        return

    if (genotypes==None) | updateAll: 

        genotypes = genimpro.genotypesForSequences(phenotypes)
        genimpro.recordings.updateGenotypesForTrack(track,sequences,genotypes)        
        


In [7]:
def analyseGenotypes(track,sessionID):
    cellBoundaries = genimpro.recordings.getSequencesForTrack(track)
    phenotypes = genimpro.recordings.getPhenotypesForTrack(track)
    if type(phenotypes) <> type(None):
        genotypes = genimpro.genotypesForCells(phenotypes,sessionID)
        genimpro.recordings.updateGenotypesForTrack(track,cellBoundaries,genotypes)

In [8]:
def addRelations(track):
    sequences = genimpro.recordings.getSequencesForTrack(track)
    (genotypes,sequenceData) = genimpro.recordings.prepareDataForRelations(track[3])

    relations = genimpro.calculateRelationsForGenotypes(genotypes,sequenceData)
    relations = relations[0:len(sequences)]
    genimpro.recordings.updateRelationsForTrack(track,sequences,relations)


In [9]:
def analyseRecording(recordingID,sessionID,updateAll=0):

    recordingDetails = genimpro.recordings.getRecordingDetails(recordingID)
    print "Analysing ID %i: %s on %s:" % (recordingDetails[0],recordingDetails[1],recordingDetails[6])
    
    for track in recordingDetails[4]:
        analysePhenotypes(track)


In [10]:
def analyseGenetics(recordingID,sessionID,updateAll=0):

    recordingDetails = genimpro.recordings.getRecordingDetails(recordingID)
    
    for track in recordingDetails[4]:
        analyseGenotypes(track,sessionID)
    
    for track in recordingDetails[4]:
        addRelations(track)

In [14]:
import aifc
track = genimpro.recordings.getRecordingDetails(133)[4][0]
audio = genimpro.recordings.getAudioForTrack(track,normalize=1)


filename = "soundcells/" + str(track[4])+".aiff"
b=aifc.open(filename,"w")
b.setnchannels(1)
b.setsampwidth(1)
b.setframerate(44100)
packed_channel = struct.pack(str(len(audio))+'h',*audio)
b.writeframes(audio)
b.close()

In [21]:
track = genimpro.recordings.getRecordingDetails(133)[4][0]



<open file 'soundcells/133.aiff', mode 'wb+' at 0x10cf03300>

In [31]:
import pydub
import cStringIO
def analysePhenotypes(track):

    #audio = genimpro.recordings.getAudioForTrack(track,normalize=1)
    
    print track[4]
    cellBoundaries = genimpro.recordings.getSequencesForTrack(track)
    
    if len(cellBoundaries)>4:
        cellIdx = 4

        audio = pydub.AudioSegment.from_file(track[2],"aiff")
        filename = "soundcells/" + str(track[4])+".aiff"
        audioslice=audio[(cellBoundaries[cellIdx][0]/samplerate)*1000:(cellBoundaries[cellIdx][1]/samplerate)*1000]+5
        audioslice.export(filename, format="aiff")

        if 1==0:
            t_silence = track[6]
            print t_silence

            minGapSize = track[7]
            print minGapSize

            cellBoundaries = genimpro.grouping.groupBySilence(audio,minGapSize=minGapSize,t_silence=t_silence)

            genimpro.recordings.updateCellsForTrack(track,cellBoundaries)

            phenotypes = genimpro.phenotypesForCells(cellBoundaries,audio)
            genimpro.recordings.updatePhenotypesForTrack(track,cellBoundaries,phenotypes)

In [24]:
def analyseSession(sessionID):
    recordings = genimpro.recordings.listRecodings(sessionID)
    for recordingID in recordings:
        analyseRecording(recordingID,sessionID)
        
    #genimpro.genomeForSession(sessionID)
    
    #for recordingID in recordings:
    #    analyseGenetics(recordingID,sessionID)

In [ ]:
import sqlite3 as lite

def deleteRecording(recordingID):
    db = lite.connect('genImpro.db')
    c = db.cursor() 
    
    sqlcommand = "delete from tracks where recordingID = %i" % (recordingID)
    c.execute(sqlcommand)
    db.commit()
    
    sqlcommand = "delete from recordings where ID = %i" % (recordingID)
    c.execute(sqlcommand)
    db.commit()    

In [32]:
analyseSession(3)

Analysing ID 139: Blind Date on 2015-10-17 20:37:16:
139
139
Analysing ID 140: Blind Date on 2015-10-17 20:25:11:
140
140
Analysing ID 141: Blind Date on 2015-10-17 20:17:42:
141
141
Analysing ID 142: Blind Date on 2015-10-17 20:47:12:
142
142
Analysing ID 143: Blind Date on 2015-10-17 16:20:18:
143
143
Analysing ID 144: Blind Date on 2015-10-17 14:54:06:
144
144
Analysing ID 145: Blind Date on 2015-10-17 20:23:45:
145
145
Analysing ID 146: Blind Date on 2015-10-17 16:20:34:
146
146
Analysing ID 147: Blind Date on 2015-10-17 20:41:37:
147
147
Analysing ID 149: Blind Date on 2015-12-29 16:49:56:
149
149
Analysing ID 150: Blind Date on 2015-10-17 20:54:40:
150
150
Analysing ID 151: Blind Date on 2015-10-17 20:19:39:
151
151
Analysing ID 152: Blind Date on 2015-10-17 20:45:29:
152
152
Analysing ID 153: Blind Date on 2015-10-17 14:58:57:
153
153
Analysing ID 154: Blind Date on 2015-10-17 16:21:10:
154
154
Analysing ID 155: Blind Date on 2015-10-17 20:49:40:
155
155
Analysing ID 156: Blind 